In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from models.unet import UNet

import numpy as np
import torch

from utils.dataset_full import LandmarksDataset, ToTensor

In [3]:
test_lungs = open("test_images_lungs.txt",'r').read().splitlines()
test_heart = open("test_images_heart.txt",'r').read().splitlines()

test_mont = [image for image in test_lungs if "MCU" in image]
test_shen = [image for image in test_lungs if "CHN" in image]
test_jsrt = [image for image in test_heart if "JP" in image]
test_pad = [image for image in test_heart if not "JP" in image]

dataset_mont = LandmarksDataset(test_mont,
                           img_path = "../Chest-xray-landmark-dataset/Images",
                           label_path = "../Chest-xray-landmark-dataset/landmarks",
                           organ = 'L',
                           transform = ToTensor()
                           )

dataset_shen = LandmarksDataset(test_shen,
                           img_path = "../Chest-xray-landmark-dataset/Images",
                           label_path = "../Chest-xray-landmark-dataset/landmarks",
                           organ = 'L',
                           transform = ToTensor()
                           )


dataset_jsrt = LandmarksDataset(test_jsrt,
                           img_path = "../Chest-xray-landmark-dataset/Images",
                           label_path = "../Chest-xray-landmark-dataset/landmarks",
                           organ = 'LHC',
                           transform = ToTensor()
                           )

dataset_pad = LandmarksDataset(test_pad,
                           img_path = "../Chest-xray-landmark-dataset/Images",
                           label_path = "../Chest-xray-landmark-dataset/landmarks",
                           organ = 'LH',
                           transform = ToTensor()
                           )

device = 'cuda:0'

In [4]:
unet_full_L = UNet(n_classes=2).to(device)
unet_full_L.load_state_dict(torch.load("weights/UNet/UNet_L_BOTH/bestDice.pt", map_location=device))
unet_full_L.eval()

unet_full_LH = UNet(n_classes=3).to(device)
unet_full_LH.load_state_dict(torch.load("weights/UNet/UNet_LH_FULL/bestDice.pt", map_location=device))
unet_full_LH.eval()

unet_full_LHC = UNet(n_classes=4).to(device)
unet_full_LHC.load_state_dict(torch.load("weights/UNet/UNet_LHC_FULL/bestDice.pt", map_location=device))
unet_full_LHC.eval()

unet_LH = UNet(n_classes=3).to(device)
unet_LH.load_state_dict(torch.load("weights/UNet/UNet_LH_STRICT/bestDice.pt", map_location=device))
unet_LH.eval()

unet_LHC = UNet(n_classes=4).to(device)
unet_LHC.load_state_dict(torch.load("weights/UNet/UNet_LHC_STRICT/bestDice.pt", map_location=device))
unet_LHC.eval()

print('Model loaded')

Model loaded


In [5]:
model_list = [unet_full_L, unet_LH, unet_full_LH, unet_LHC, unet_full_LHC]
model_names = ['L (Full)', 'LH (Strict)', 'LH (Full)', 'LHC (Strict)', 'LHC (Full)']

datasets = [dataset_mont,dataset_shen,dataset_jsrt, dataset_pad]
dataset_names = ["Montgomery (L)", "Shenzhen (L)", "JSRT (LHC)", "Padchest (LH)"]

In [6]:
from medpy.metric import dc, hd
import cv2

def getDenseMask(RL, LL, H = None, CLA1 = None, CLA2 = None, imagesize = 1024):
    img = np.zeros([1024,1024])
    imgcla = np.zeros([1024,1024])
    
    RL = RL.reshape(-1, 1, 2).astype('int')
    LL = LL.reshape(-1, 1, 2).astype('int')

    img = cv2.drawContours(img, [RL], -1, 1, -1)
    img = cv2.drawContours(img, [LL], -1, 1, -1)
    
    if H is not None:
        H = H.reshape(-1, 1, 2).astype('int')
        img = cv2.drawContours(img, [H], -1, 2, -1)
        
    if CLA1 is not None:
        CLA1 = CLA1.reshape(-1, 1, 2).astype('int')
        img = cv2.drawContours(img, [CLA1], -1, 3, -1)
    
    if CLA2 is not None:
        CLA2 = CLA2.reshape(-1, 1, 2).astype('int')
        img = cv2.drawContours(img, [CLA2], -1, 3, -1)
    
    return img

def evalImageMetrics(output, target):
    dcp = dc(output == 1, target == 1)

    try:
        dcc = dc(output == 2, target == 2)
    except:
        dcc = -1
    
    try:
        dccla = dc(output == 3, target == 3)
    except:
        dccla = -1

    hdp = hd(output == 1, target == 1)    
    
    try:
        hdc = hd(output == 2, target == 2)
    except:
        hdc = -1

    try:
        hdcla = hd(output == 3, target == 3)
    except:
        hdcla = -1
            
    return [dcp, dcc, dccla, hdp, hdc, hdcla]

In [7]:
import pandas as pd

results = pd.DataFrame()

for k in range(0,4):
    dataset = datasets[k]

    for i in range(0, len(dataset.images)):
        print('\r',dataset_names[k], i+1,'of', len(dataset.images),end='')
        with torch.no_grad():
            sample = dataset[i]

            data, target = sample['image'], sample['landmarks']
            data = torch.unsqueeze(data, 0).to(device)
            target =  target.reshape(-1,2).numpy()
            
            RL = target[:44] * 1024
            LL = target[44:94] * 1024

            if k > 1:
                H = target[94:120] * 1024
            else:
                H = None
            
            if k == 2:
                CLA1 = target[120:143] * 1024
                CLA2 = target[143:] * 1024
            else:
                CLA1 = None
                CLA2 = None

            targetseg = getDenseMask(RL, LL, H, CLA1, CLA2)

            for j in range(0, len(model_list)):
                output = model_list[j](data)
                outseg = torch.argmax(output[0,:,:,:], axis = 0).cpu().numpy()

                metrics = evalImageMetrics(outseg, targetseg)

                aux = pd.DataFrame([[i, dataset_names[k], model_names[j]] + metrics], 
                                     columns=['i', 'Dataset', 'Model', 
                                     'Dice Lungs','Dice Heart','Dice Cla','HD Lungs','HD Heart','HD Cla'])
                results = results.append(aux, ignore_index = True)
        print('')

 Padchest (LH) 27 of 27

In [8]:
for d in dataset_names:
    print(d)
    sub = results[results['Dataset'] == d]
    del sub['i']
    if '(L)' in d:
        sub = sub[['Dice Lungs', 'HD Lungs', 'Model']]
    elif '(LH)' in d:
        sub = sub[['Dice Lungs', 'HD Lungs', 'Dice Heart', 'HD Heart', 'Model']]
    elif '(LHC)' in d:
        sub = sub[['Dice Lungs', 'HD Lungs', 'Dice Heart', 'HD Heart', 'Dice Cla', 'HD Cla', 'Model']]

    group = sub.groupby('Model').mean()
    display(group)#.iloc[[0,2,1,4,3],:])


Montgomery (L)


,Dice Lungs,HD Lungs
Model,,
L (Full),0.976844,53.305432
LH (Full),0.973885,69.974305
LH (Strict),0.960535,73.395095
LHC (Full),0.973483,72.506772
LHC (Strict),0.901322,182.352712


Shenzhen (L)


,Dice Lungs,HD Lungs
Model,,
L (Full),0.967040,48.239872
LH (Full),0.964500,73.956087
LH (Strict),0.956960,131.195306
LHC (Full),0.965583,72.674117
LHC (Strict),0.911196,217.629522


JSRT (LHC)


,Dice Lungs,HD Lungs,Dice Heart,HD Heart,Dice Cla,HD Cla
Model,,,,,,
L (Full),0.949117,99.873489,0.000000,-1.000000,0.000000,-1.000000
LH (Full),0.950301,103.812785,0.939463,51.788766,0.000000,-1.000000
LH (Strict),0.950118,101.087653,0.943665,51.667169,0.000000,-1.000000
LHC (Full),0.975339,63.024350,0.938867,54.460874,0.913340,49.237748
LHC (Strict),0.974634,67.454422,0.936825,67.584456,0.932082,44.843538


Padchest (LH)


,Dice Lungs,HD Lungs,Dice Heart,HD Heart
Model,,,,
L (Full),0.956839,85.154843,0.000000,-1.000000
LH (Full),0.957829,94.834847,0.889956,75.221953
LH (Strict),0.959333,89.351454,0.933086,80.477407
LHC (Full),0.955580,106.083443,0.908984,76.740715
LHC (Strict),0.885634,227.134293,0.869926,220.792234
